In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as plt
import re
from pylab import rcParams
rcParams['figure.figsize'] = 13,6
data = pd.read_csv('Netology_A_B-testing_HW_1.csv')
pd.options.display.max_rows = 1000

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101500 entries, 0 to 101499
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         101500 non-null  int64  
 1   date               101500 non-null  object 
 2   user_id            101500 non-null  int64  
 3   hour               101500 non-null  int64  
 4   app_opened         101500 non-null  int64  
 5   price_seen         101500 non-null  int64  
 6   order_made         101500 non-null  int64  
 7   surge              91431 non-null   object 
 8   ride_completed     101500 non-null  int64  
 9   user_cancelled     101500 non-null  int64  
 10  age                101500 non-null  int64  
 11  os                 101500 non-null  object 
 12  city_center_order  101500 non-null  int64  
 13  order_class        101500 non-null  object 
 14  distance           91431 non-null   float64
 15  rfm                101500 non-null  object 
dtypes:

In [5]:
for col in data.columns:
    pct_missing = data[col].isna().mean()
    print(f'{col} - {pct_missing :.1%}')

Unnamed: 0 - 0.0%
date - 0.0%
user_id - 0.0%
hour - 0.0%
app_opened - 0.0%
price_seen - 0.0%
order_made - 0.0%
surge - 9.9%
ride_completed - 0.0%
user_cancelled - 0.0%
age - 0.0%
os - 0.0%
city_center_order - 0.0%
order_class - 0.0%
distance - 9.9%
rfm - 0.0%


In [6]:
data.describe()

,Unnamed: 0,user_id,hour,app_opened,price_seen,order_made,ride_completed,user_cancelled,age,city_center_order,distance
count,101500.000000,101500.000000,101500.000000,101500.0,101500.000000,101500.000000,101500.000000,101500.000000,101500.000000,101500.000000,91431.000000
mean,50749.500000,549874.802207,11.481429,1.0,0.900798,0.731389,0.620365,0.111025,40.471764,0.571675,5.371152
std,29300.670499,258600.104176,6.917522,0.0,0.298934,0.443239,0.485299,0.314164,13.536322,0.494838,4.118531
min,0.000000,100093.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,18.000000,0.000000,0.010946
25%,25374.750000,328223.000000,5.000000,1.0,1.000000,0.000000,0.000000,0.000000,29.000000,0.000000,2.438335
50%,50749.500000,548827.000000,11.000000,1.0,1.000000,1.000000,1.000000,0.000000,40.000000,1.000000,4.279565
75%,76124.250000,773051.000000,17.000000,1.0,1.000000,1.000000,1.000000,0.000000,52.000000,1.000000,7.129814
max,101499.000000,999978.000000,23.000000,1.0,1.000000,1.000000,1.000000,1.000000,69.000000,1.000000,40.268966


In [7]:
"""
Явных выбросов не обнаружил
"""

'\nЯвных выбросов не обнаружил\n'

In [8]:
data_nan = data.loc[data['surge'].isnull()] #Оставил строки, где surge nan
#data_nan

In [9]:
data_nan_order = data_nan.loc[data_nan['price_seen'] !=0] #Проверил, что во всех строках price_seen равен 0
data_nan_order

,Unnamed: 0,date,user_id,hour,app_opened,price_seen,order_made,surge,ride_completed,user_cancelled,age,os,city_center_order,order_class,distance,rfm


In [10]:
"""
Nan в surge только, если клиент не ввел данные маршрута и не сделал заказ
Разобью таблицу на две - с оформленными заказами и неоформленными. 
"""

'\nNan в surge только, если клиент не ввел данные маршрута и не сделал заказ\nРазобью таблицу на две - с оформленными заказами и неоформленными. \n'

In [11]:
data_order = data.dropna() #Удалил nan
data_order

,Unnamed: 0,date,user_id,hour,app_opened,price_seen,order_made,surge,ride_completed,user_cancelled,age,os,city_center_order,order_class,distance,rfm
0,0,2020-05-21,867689,12,1,1,1,no surge,1,0,20,iOS,0,business,7.982135,low
1,1,2020-05-23,752172,5,1,1,1,no surge,1,0,37,Android,1,economy,2.908468,high
2,2,2020-05-20,486559,15,1,1,1,no surge,1,0,47,Android,0,comfort,7.224614,low
3,3,2020-05-19,304024,0,1,1,1,no surge,1,0,59,Android,1,economy,1.874349,low
4,4,2020-05-23,139420,0,1,1,1,no surge,1,0,19,Android,0,business,10.704778,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101495,101495,2020-05-24,288445,16,1,1,1,no surge,1,0,29,Android,0,comfort,1.725907,high
101496,101496,2020-05-19,627754,2,1,1,1,surge,1,0,23,iOS,1,comfort,6.206134,medium
101497,101497,2020-05-23,784442,1,1,1,1,no surge,1,0,40,iOS,0,economy,6.624225,medium
101498,101498,2020-05-20,232993,13,1,1,1,no surge,1,0,52,iOS,0,business,8.825188,low


In [27]:
data.groupby(['age', 'app_opened']).agg({'Unnamed: 0': 'count'})

,,Unnamed: 0
age,app_opened,
18,1,2236
19,1,2312
20,1,2155
21,1,2237
22,1,2269
23,1,2228
24,1,2249
25,1,2212
26,1,2187
